In [1]:
# !wget https://github.com/UMxYTL-AI-Labs/MalayMMLU/raw/refs/heads/main/data/MalayMMLU_0shot.json

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
from transformers import GptOssForCausalLM, AutoModelForCausalLM, AutoTokenizer, Mxfp4Config
from peft import PeftModel
import torch
torch.set_grad_enabled(False)

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

/root/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [4]:
# !pip3 install accelerate kernels

In [5]:
from glob import glob
from transformers.trainer_utils import get_last_checkpoint

folders = glob('malaysian-reasoning-20b-lora-r*')
folders = [get_last_checkpoint(f) for f in folders if 'merge' not in f]
folders

['malaysian-reasoning-20b-lora-r256/checkpoint-1146',
 'malaysian-reasoning-20b-lora-r64/checkpoint-1146',
 'malaysian-reasoning-20b-lora-r16/checkpoint-1146',
 'malaysian-reasoning-20b-lora-r32/checkpoint-1146']

In [6]:
for folder in folders:
    model_kwargs = dict(
        attn_implementation="kernels-community/vllm-flash-attn3",
        torch_dtype=torch.bfloat16, 
        use_cache=True, 
        device_map='auto',
        # quantization_config=quantization_config,
    )
    model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs).cuda()
    peft_model_id = folder
    model = PeftModel.from_pretrained(model, peft_model_id)
    model = model.merge_and_unload()

    model.save_pretrained(f'{os.path.split(folder)[0]}-merged')
    tokenizer.save_pretrained(f'{os.path.split(folder)[0]}-merged')

    del model

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.34s/it]
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.19s/it]
